In [1]:
from image_ocr import extract_text
from image_vector import read_img, reduce_dimensionality, load_model
import torch
import numpy as np
import pickle
import time
import cv2
import re
import os
import easyocr
from sklearn.metrics.pairwise import cosine_similarity
from ocr_netvlad import load_image_labels,get_image_point,find_most_similar_images

c:\Users\jin03\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
if __name__ == '__main__':

    start_time = time.time()

    weight_path = 'pretrain-model.pth.tar'
    cuda = torch.cuda.is_available()
    model = load_model(weight_path, cuda)
    reader = easyocr.Reader(['ko', 'en'])

    labels = load_image_labels('image_label.txt')
    label2 = load_image_labels('test_image_label.txt')

    target_image = 'test_image/test(64).jpg'
    target_image2 = target_image.replace('test_image/', '')
    point = get_image_point(target_image2, label2)
    print(f"'{target_image}'의 포인트 값: {point}")

    print()
    r = find_most_similar_images(target_image, model,reader,labels, top_k=5, ocr_data='reduced_ocr_text_index.pkl', pca_path='pca_model.pkl', data_path='reduced_image_vectors_list.pkl', cuda=cuda)
    total_time = time.time() - start_time
    print(r)
    print(f"Total execution time: {total_time:.4f} seconds")

In [2]:
import os
import time
import torch
import easyocr

def get_floor_from_point(point):
    """포인트 값에서 층수를 결정하는 함수"""
    if point == 2:  
        return "2F"
    elif point == 3: 
        return "3F"
    else:
        return None

if __name__ == '__main__':

    start_time = time.time()

    weight_path = 'pretrain-model.pth.tar'
    cuda = torch.cuda.is_available()
    model = load_model(weight_path, cuda)
    reader = easyocr.Reader(['ko', 'en'])

    labels = load_image_labels('image_label.txt')
    label2 = load_image_labels('test_image_label.txt')

    test_folder = 'test_image'  
    test_images = [f for f in os.listdir(test_folder) if f.endswith('.jpg')]  

    correct_total = 0
    total_count = 0
    correct_no_floor = 0
    floor_correct = 0
    top = 1

    for test_image in test_images:
        # 각 이미지에 대한 포인트 값 및 유사도 계산
        target_image_path = os.path.join(test_folder, test_image)
        target_image_name = test_image
        point = get_image_point(target_image_name, label2)
        floor = get_floor_from_point(point[2])  # 포인트 값으로 층수 추출
        print(f"\n'{target_image_name}'의 포인트 값: {point}, 층수: {floor}")

        # 유사한 이미지 검색 
        most_similar_images = find_most_similar_images(
            target_image_path, model, reader, labels, top_k=top, 
            ocr_data='reduced_ocr_text_index.pkl', pca_path='pca_model.pkl', 
            data_path='reduced_image_vectors_list.pkl', cuda=cuda
        )

        total_count += 1
        cnf = 0
        cf = 0
        for image_name, similarity, image_point, _ in most_similar_images:
            # 전체 적중률 계산
    
            if image_point[2] == point[2]:
                cf += 1
                if (image_point[0] == point[0] and image_point[1] == point[1]):
                    cnf += 1
                    correct_total += 1
                    print('위치 적중', total_count,"중",correct_total,"개")
                    break
            else:
                if (image_point[0] == point[0] and image_point[1] == point[1]):
                    cnf += 1

        if cnf >= 1 :
            correct_no_floor += 1
        if cf >= 1 :
            floor_correct += 1

    # 전체 적중률 계산
    total_accuracy = correct_total / total_count if total_count > 0 else 0
    print(f"\n전체 적중률: {total_accuracy:.2%}")
    print(f"{top}개의 유사도로 비교 시 {total_count}개의 이미지 중 {correct_total}개 일치")

    # 좌표 적중률 계산
    nofloor_accuracy = correct_no_floor / total_count if total_count > 0 else 0
    print(f"\n좌표 적중률: {nofloor_accuracy:.2%}")
    print(f"{top}개의 유사도로 비교 시 {total_count}개의 이미지 중 {correct_no_floor}개 일치")

    # 층 적중률 계산
    floor_accuracy = floor_correct / total_count if total_count > 0 else 0
    print(f"\n층 적중률: {floor_accuracy:.2%}")
    print(f"{top}개의 유사도로 비교 시 {total_count}개의 이미지 중 {floor_correct}개 일치")

    total_time = time.time() - start_time
    print(f"Total execution time: {total_time:.4f} seconds")


c:\Users\jin03\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jin03\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



'test(1).jpg'의 포인트 값: (0.0, 0.0, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산
위치 적중 1 중 1 개

'test(10).jpg'의 포인트 값: (0.0, 0.0, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산

'test(100).jpg'의 포인트 값: (15.84, 34.38, 2.0), 층수: 2F
층 구별 실패 및 OCR 성공, 전체 이미지에서 OCR,유사도 계산
위치 적중 3 중 2 개

'test(101).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
OCR로 매칭된 이미지 존재, 코사인 유사도 계산 시작
위치 적중 4 중 3 개

'test(102).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
OCR로 매칭된 이미지 존재, 코사인 유사도 계산 시작
위치 적중 5 중 4 개

'test(103).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산

'test(104).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산

'test(105).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산
위치 적중 8 중 5 개

'test(106).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산
위치 적중 9 중 6 개

'test(107).jpg'의 포인트 값: (26.28, 15.84, 2.0), 층수: 2F
층 구별 및 OCR 모두 실패, 전체 이미지에서 유사도 계산
위치 적중 10 중 7 개

'test(108).jpg'의 포인트 값: (26.28, 15.84, 2.0)